In [354]:
import pandas as pd
import re
import numpy as np

In [355]:
df = pd.read_csv("preprocessed_data_craigslist.csv")
df = df.iloc[:,1:]
df

,price,location,url,date,title,numimage,text,condition,makemanufacturer,modelnamenumber,...,electricassist,framesize,handlebartype,suspension,wheelsize,sizedimensions,serialnumber,paintcolor,yearmanufactured,days
0,1000.0,auburn,https://auburn.craigslist.org/mcd/d/fayettevil...,2022-04-05 15:32,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",10.0,"WE SHIP NATIONWIDE, FINANCE NATIONWIDE! YOU SE...",unknown,0.0,0.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,19
1,1.0,auburn,https://auburn.craigslist.org/for/d/defuniak-s...,2022-04-17 13:32,Atv’s,23.0,"Used and New atv’s , go karts and dirt bikes f...",unknown,0.0,0.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,8
2,8795.0,auburn,https://auburn.craigslist.org/fod/d/fairburn-2...,2022-04-16 14:51,2017 Club Car Precedent 4 Seater Gas Alabama C...,20.0,Very Nice 2017 Club Car Precedent EFI Gas Alab...,4,0.0,0.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,9
3,30.0,auburn,https://auburn.craigslist.org/bop/d/opelika-th...,2022-04-16 09:07,Thule 961XT Speedway Bike Strap Rear Rack Carrier,6.0,Thule 961XT Speedway - Bike Strap Rear Rack Ca...,unknown,1.0,1.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,9
4,112.0,auburn,https://auburn.craigslist.org/sgd/d/full-garag...,2022-04-15 12:52,"Full Garage Gym-Squat Rack, Dumbbells- Financi...",4.0,"Full Garage Gym Setup- Squat Rack, Adjustable ...",4,1.0,1.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,75.0,seattle,https://seattle.craigslist.org/oly/bik/d/olymp...,2022-03-28 06:39,Bikes that must go.. take what ever,2.0,I have like 6 bikes that have to go today Mar...,unknown,1.0,1.0,...,0.0,3,unknown,0.0,26,0,0,0,0,28
3581,100.0,seattle,https://seattle.craigslist.org/oly/bik/d/olymp...,2022-03-28 06:40,I got three bikes that have to go asap!!!,2.0,Different condition bikes that need to go.. le...,2,1.0,1.0,...,0.0,3,unknown,0.0,24,0,1,0,0,28
3582,110.0,seattle,https://seattle.craigslist.org/kit/bop/d/breme...,2022-03-27 13:55,Bike Carrier 4 bikes,10.0,"Land Rover for 2"" receiver, carries 4 bikes or...",3,1.0,0.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,29
3583,50.0,seattle,https://seattle.craigslist.org/skc/bop/d/kent-...,2022-03-27 07:57,Gravity rack for two bikes,3.0,Save space with this heavy duty bike rack for ...,unknown,0.0,0.0,...,0.0,unknown,unknown,0.0,unknown,0,0,0,0,29


In [356]:
for column in df.columns:
    print(column,df[column].dtype)

price float64
location object
url object
date object
title object
numimage float64
text object
condition object
makemanufacturer float64
modelnamenumber float64
bicycleframematerial object
bicycletype object
braketype object
electricassist float64
framesize object
handlebartype object
suspension float64
wheelsize object
sizedimensions int64
serialnumber int64
paintcolor int64
yearmanufactured int64
days int64


In [357]:
important_attributes = ['price', 'condition', 'bicycleframematerial', 'bicycletype', 'braketype', 'framesize', 'handlebartype','wheelsize']

for column in important_attributes:
    df_values = pd.DataFrame(df.groupby([column]).size())
    display(df_values)

,0
price,
1.0,171
2.0,1
4.0,1
5.0,16
7.0,1
...,...
139572.0,1
185746.0,1
445522.0,1


,0
condition,
1,10
2,53
3,344
4,1203
unknown,1975


,0
bicycleframematerial,
alloy,162
aluminum,513
carbon fiber,84
composite,12
scandium,1
steel,425
titanium,8
unknown,2380


,0
bicycletype,
bmx,164
cargo/pedicab,6
cruiser,144
cyclocross,4
folding,34
gravel,23
hybrid/comfort,116
kids,186
mountain,369


,0
braketype,
hydraulic,156
mechanical,85
special,268
unknown,3076


,0
framesize,
1,182
2,396
3,522
unknown,2485


,0
handlebartype,
special,342
unknown,3243


,0
wheelsize,
10,18
12,49
14,25
16,68
18,34
20,184
24,90
25,14
26,372


In [358]:
def loss_analysis(df):
    unknown_dic = dict()
    n=len(df)
    for column in important_attributes:
        df_values = pd.DataFrame(df.groupby([column]).size())
        if column == 'price':
            unknown = len(df[(df['price']<10) | (df['price']>2500)])
            unknown_dic[column]=[unknown,round(unknown/n*100,1)]
        try:
            unknown = int(df_values.loc['unknown'])
        except:
            pass
        else:
            unknown_dic[column]=[unknown,round(unknown/n*100,1)]
    df_unknown = pd.DataFrame(data=unknown_dic,index=['Unknowns','% of Unknowns']).transpose()
    return df_unknown

In [359]:
loss_analysis(df)

,Unknowns,% of Unknowns
price,1091.0,30.4
condition,1975.0,55.1
bicycleframematerial,2380.0,66.4
bicycletype,2292.0,63.9
braketype,3076.0,85.8
framesize,2485.0,69.3
handlebartype,3243.0,90.5
wheelsize,2371.0,66.1


In [360]:
def retrieve_condition_from_txt(text):
    sentences = re.compile(r'([A-Z][^\.!?]*[\.!?])', re.M).findall(text)
    for sentence in sentences:
        if 'condition' in sentence:
            return sentence

def fix_condition_from_txt(text,condition):
    if condition != 'unknown':
        return condition
    else:
        sentence = retrieve_condition_from_txt(text)
        if sentence is None:
            return 'unknown'
        else:
            sentence = sentence.lower()
            if 'very good' in sentence or 'excellent' in sentence or 'new' in sentence or 'very nice' in sentence:
                return str(4)
            if 'good used condition' in sentence or 'good' in sentence or 'very pretty' in sentence or 'great' in sentence:
                return str(3)
            if 'nice' in sentence or 'fair' in sentence:
                return str(2)
            if 'poor' in sentence or 'salvage' in sentence:
                return str(1)
            else:
                return 'unknown'

before = loss_analysis(df).loc['condition','Unknowns']
df['condition'] = df.apply(lambda x: fix_condition_from_txt(x['text'],x['condition']), axis=1)
after = loss_analysis(df).loc['condition','Unknowns']
print("+ "+ str(int(before-after)) + " data for 'condition' after using fix_condition_from_txt.")


+ 246 data for 'condition' after using fix_condition_from_txt.


In [361]:
def fix_material_from_txt(text, material):
    if material != 'unknown':
        return material
    else:
        if 'aluminum' in text:
            return 'aluminum'
        if 'alloy' in text:
            return 'alloy'
        if 'steel' in text:
            return 'steel'
        if 'carbon' in text:
            return 'carbon fiber'
        if 'titanium' in text:
            return 'titanium'
        if 'composite' in text:
            return 'composite'
        if 'scandium' in text:
            return 'scandium'
        else:
            return 'unknown'


before = loss_analysis(df).loc['bicycleframematerial','Unknowns']
df['bicycleframematerial'] = df.apply(lambda x: fix_material_from_txt(x['text'],x['bicycleframematerial']), axis=1)
after = loss_analysis(df).loc['bicycleframematerial','Unknowns']
print("+ "+ str(int(before-after)) + " data for 'bicycleframematerial' after using fix_material_from_txt.")

+ 255 data for 'bicycleframematerial' after using fix_material_from_txt.


In [362]:
def fix_bicycletype_from_txt(text,bicycletype):
    if bicycletype != 'unknown':
        return bicycletype
    else:
        if 'mountain' in text:
            return 'mountain'
        if 'kids' in text:
            return 'kids'
        if 'hybrid' in text:
            return 'hybrid/comfort'
        if 'comfort' in text:
            return 'hybrid/comfort'
        if 'cruiser' in text:
            return 'cruiser'
        if 'bmx' in text:
            return 'bmx'
        if 'road' in text:
            return 'road'
        if 'unicycle' in text:
            return 'unicycle'
        if 'folding' in text:
            return 'folding'
        if 'gravel' in text:
            return 'gravel'
        if 'recumbent' in text:
            return 'recumbent/trike'
        if 'cyclocross' in text:
            return 'cyclocross'
        if 'tandem' in text:
            return 'tandem'
        if 'track' in text:
            return 'track'
        if 'cargo' in text:
            return 'cargo/pedicab'
        if 'pedicab' in text:
            return 'cargo/pedicab'
        else:
            return 'unknown'

before = loss_analysis(df).loc['bicycletype','Unknowns']
df['bicycletype'] = df.apply(lambda x: fix_bicycletype_from_txt(x['text'],x['bicycletype']), axis=1)
after = loss_analysis(df).loc['bicycletype','Unknowns']
print("+ "+ str(int(before-after)) + " data for 'bicycletype' after using fix_bicycletype_from_txt.")

+ 791 data for 'bicycletype' after using fix_bicycletype_from_txt.


In [363]:
def fix_braketype_from_txt(text,braketype):
    if braketype != 'unknown':
        return braketype
    else:
        if 'mechanical' in text:
            return 'mechanical'
        if 'hydraulic' in text:
            return 'hydraulic'
        else:
            return 'unknown'

before = loss_analysis(df).loc['braketype','Unknowns']
df['braketype'] = df.apply(lambda x: fix_braketype_from_txt(x['text'],x['braketype']), axis=1)
after = loss_analysis(df).loc['braketype','Unknowns']
print("+ "+ str(int(before-after)) + " data for 'braketype' after using fix_braketype_from_txt.")

+ 85 data for 'braketype' after using fix_braketype_from_txt.


In [364]:
loss_analysis(df)

,Unknowns,% of Unknowns
price,1091.0,30.4
condition,1729.0,48.2
bicycleframematerial,2125.0,59.3
bicycletype,1501.0,41.9
braketype,2991.0,83.4
framesize,2485.0,69.3
handlebartype,3243.0,90.5
wheelsize,2371.0,66.1
